In [ ]:
from omni.isaac.core.utils.stage import add_reference_to_stage
from isaacsim.core.api import World
import omni.usd
from omni.isaac.core.prims import XFormPrim
from pxr import Sdf, Gf, Usd, UsdGeom, UsdShade, Vt
import omni.isaac.core.utils.stage as stage_utils # add_reference_to_stage / add_ground_plane
import omni.kit.commands

# make GRIDMAP and Dataset

In [ ]:
import typing

import carb
import omni.usd

# nvidia 에서 제공하는 bbox 계산 코드
def compute_path_bbox(prim_path: str) -> typing.Tuple[carb.Double3, carb.Double3]:
    """
    Compute Bounding Box using omni.usd.UsdContext.compute_path_world_bounding_box
    See https://docs.omniverse.nvidia.com/kit/docs/omni.usd/latest/omni.usd/omni.usd.UsdContext.html#omni.usd.UsdContext.compute_path_world_bounding_box

    Args:
        prim_path: A prim path to compute the bounding box.
    Returns:
        A range (i.e. bounding box) as a minimum point and maximum point.
    """
    return omni.usd.get_context().compute_path_world_bounding_box(prim_path)

In [ ]:
import numpy as np
from pxr import Usd, UsdGeom, Gf
import omni.usd
from omni.isaac.core.prims import XFormPrim

# 카메라 시점 = 드론 시점 -> 그리드맵 생성
# 3d BBOX 좌표를 이용해
# 그리드맵을 생성
def generate_drone_grid_map(
    drone_prim_path: str,
    map_center=(0.0, 0.0),
    map_size=(10.0, 10.0),
    resolution=0.1
):
    stage = omni.usd.get_context().get_stage()
    if stage is None:
        print("에러: Stage를 가져오지 못했습니다.")
        return None

    size_x = int(map_size[0] / resolution)
    size_y = int(map_size[1] / resolution)
    origin_x = map_center[0] - map_size[0] * 0.5
    origin_y = map_center[1] - map_size[1] * 0.5
    grid = np.zeros((size_y, size_x), dtype=np.uint8)  # [y, x]

    if not stage.GetPrimAtPath(drone_prim_path):
        print(f"에러: Prim '{drone_prim_path}' 없음")
        return None

    drone = XFormPrim(drone_prim_path, name="drone_xform")
    pos, _rot = drone.get_world_pose()
    drone_z = float(pos[2])
    print(f"기준 고도 Z: {drone_z:.2f} m")

    purposes = [UsdGeom.Tokens.default_, UsdGeom.Tokens.render, UsdGeom.Tokens.proxy]
    bbox_cache = UsdGeom.BBoxCache(
        Usd.TimeCode.Default(), purposes,
        useExtentsHint=True, ignoreVisibility=False
    )
    scope = stage.GetPrimAtPath("/World/Scope")
    for child in scope.GetChildren():
        prim_path = str(child.GetPath())
        min_, max_ = compute_path_bbox(prim_path)
        xmin, ymin, zmin = min_
        xmax, ymax, zmax = max_
        print("Prim's Zmax coordinate",zmax)
        # if zmax <= drone_z:
        #     continue

        sx = int((xmin - origin_x) / resolution)
        ex = int((xmax - origin_x) / resolution)
        sy = int((ymin - origin_y) / resolution)
        ey = int((ymax - origin_y) / resolution)
        sx = max(0, min(size_x, sx))
        ex = max(0, min(size_x, ex))
        sy = max(0, min(size_y, sy))
        ey = max(0, min(size_y, ey))
        print(sx, ex, sy, ey)
        if sx < ex and sy < ey:
            grid[sy:ey, sx:ex] = 1

    print("--- 그리드 맵 생성 완료 ---")
    return grid


In [ ]:
# Isaac Sim Jupyter 커널
import numpy as np
from pxr import Usd, UsdGeom
import omni.usd

# /World/Scope/Person/Person1 , /World/Scope/Person/Person2 ... 형태로 스테이지를 구성
# 따라서 /World/Scope/Person 의 children 을 받아와서 사람만을 표시한 grid map 을 생성 가능
def generate_person_goal_map(
    person_root_path="/World/Scope/Person",
    map_center=(0.0, 0.0),
    map_size=(10.0, 10.0),
    resolution=0.1,
    sigma_cells=1.5
):
    stage = omni.usd.get_context().get_stage()
    if stage is None:
        print("Stage 없음"); return None

    root = stage.GetPrimAtPath(person_root_path)
    if not root.IsValid():
        print(f"Prim '{person_root_path}' 없음"); return None

    size_x = int(map_size[0] / resolution)
    size_y = int(map_size[1] / resolution)
    origin_x = map_center[0] - map_size[0] * 0.5
    origin_y = map_center[1] - map_size[1] * 0.5
    grid = np.zeros((size_y, size_x), dtype=np.uint8)

    # BBoxCache는 한 번만 생성
    cache = UsdGeom.BBoxCache(
        Usd.TimeCode.Default(),
        [UsdGeom.Tokens.default_, UsdGeom.Tokens.render, UsdGeom.Tokens.proxy],
        useExtentsHint=True, ignoreVisibility=False
    )

    def world_to_grid(x, y):
        j = int((x - origin_x) / resolution)  # col
        i = int((y - origin_y) / resolution)  # row
        if 0 <= i < size_y and 0 <= j < size_x:
            return i, j
        return None

    def draw_gaussian(i0, j0, sigma):
        H, W = grid.shape
        r = int(3*sigma)
        i_min, i_max = max(0, i0-r), min(H-1, i0+r)
        j_min, j_max = max(0, j0-r), min(W-1, j0+r)
        ys = np.arange(i_min, i_max+1)[:, None]
        xs = np.arange(j_min, j_max+1)[None, :]
        g = np.exp(-((ys - i0)**2 + (xs - j0)**2)/(2*sigma*sigma))
        grid[i_min:i_max+1, j_min:j_max+1] = np.maximum(grid[i_min:i_max+1, j_min:j_max+1], g)

    # ✅ 루트 바로 아래 자식만 처리 (Person_01, Person_02, …)
    #    루트 자신(Scope)도, 하위 파츠(메시/본 조각)도 건들지 않음.
    persons = []
    for child in root.GetChildren():
        name = child.GetName()
        # 이름 규칙 필터(원하면 제거 가능)
        if not name.startswith("Person"):
            continue
        # 자식 전체를 대표하는 bbox (child 한 개에 대해 월드 bbox 계산)
        try:
            wb = cache.ComputeWorldBound(child)
            ar = wb.ComputeAlignedRange()
            mn, mx = ar.GetMin(), ar.GetMax()
            xmin, ymin = float(mn[0]), float(mn[1])
            xmax, ymax = float(mx[0]), float(mx[1])
            # 비정상 bbox 거르기
            if (xmax - xmin) < 1e-4 and (ymax - ymin) < 1e-4:
                continue
            persons.append(((xmin, ymin), (xmax, ymax)))
        except Exception:
            continue

    # 각 사람의 (x,y) 센터만 히트맵으로 찍기
    for (xmin, ymin), (xmax, ymax) in persons:
        x_c = 0.5*(xmin + xmax)
        y_c = 0.5*(ymin + ymax)
        ij = world_to_grid(x_c, y_c)
        if ij is not None:
            draw_gaussian(ij[0], ij[1], sigma_cells)

    grid = np.clip(grid, 0.0, 1.0)
    print(f"--- 사람 goal 히트맵 생성 완료 (top-level persons: {len(persons)}) ---")
    return grid


In [ ]:
import asyncio
import omni.replicator.core as rep
import numpy as np
rep.settings.set_render_pathtraced()

from pathlib import Path
obs_dir = Path("./project/grid_map")
person_dir = Path("./project/grid_map_person")
obs_dir.mkdir(parents=True, exist_ok=True)
person_dir.mkdir(parents=True, exist_ok=True)
frames=5000

async def run(frames):
    DRONE_PATH = "/World/Camera"

    with rep.new_layer():
        #camera= rep.create.camera(position=(25,-15,24), look_at=(0,0,0))
        camera = rep.get.prim_at_path(DRONE_PATH)
        render_product = rep.create.render_product(camera,(1024,1024))
        # 천장뷰 카메라
        top_cam = rep.create.camera(
            position=(0, 0, 50),    # z=50 위에서
            look_at=(0, 0, 0)       # 원점 바라보기
        )
        top_rp = rep.create.render_product(top_cam, (1024, 1024))
        box10=rep.get.prim_at_path("/World/Scope/box2_4")
        box1=rep.get.prim_at_path("/World/Scope/box2_0")
        box2=rep.get.prim_at_path("/World/Scope/box2_1")
        box3=rep.get.prim_at_path("/World/Scope/box2_2")
        box4=rep.get.prim_at_path("/World/Scope/box2_3")
        box5=rep.get.prim_at_path("/World/Scope/box2_5")
        box6=rep.get.prim_at_path("/World/Scope/box2_6")
        box7=rep.get.prim_at_path("/World/Scope/box2_7")
        box8=rep.get.prim_at_path("/World/Scope/box2_8")
        box9=rep.get.prim_at_path("/World/Scope/box2_9")

        car=rep.get.prim_at_path("/World/Scope/car1")
        pail=rep.get.prim_at_path("/World/Scope/pail1")

        person1=rep.get.prim_at_path("/World/Scope/Person/Person1")
        person2=rep.get.prim_at_path("/World/Scope/Person/Person2")
        #debris = rep.get.prim_at_path("/World/Construction_Debris")
        log1 = rep.get.prim_at_path("/World/Scope/Logs")
        log2 = rep.get.prim_at_path("/World/Scope/Logs_01")
        log3 = rep.get.prim_at_path("/World/Scope/Logs_02")
        with rep.create.group([log1,log2,log3]):
            rep.modify.semantics([('class','log')])
        #with debris:
        #    rep.modify.semantics([('class','debris')])
        with rep.create.group([box1,box2,box3,box4,box5,box6,box7,box8,box9,box10]):
            rep.modify.semantics([('class','box')])
        with rep.create.group([person1, person2]):
            rep.modify.semantics([('class','person')])
        with car:
            rep.modify.semantics([('class','car')])
        with pail:
            rep.modify.semantics([('class','pail')])

        with rep.trigger.on_frame(num_frames=frames):
            with rep.create.group([box1,box2,box4,box3,box5]):
                rep.modify.pose(
                    position=rep.distribution.uniform((-5,-10,0.85),(1,2,0.9)),
                    scale=rep.distribution.uniform(5,7))

            with rep.create.group([box6,box7,box8,box9,box10]):
                rep.modify.pose(
                    position=rep.distribution.uniform((-5,-11,0.85),(13,-7,0.9)),
                    scale=rep.distribution.uniform(5,7))

            with rep.create.group([log1,log2]):
                rep.modify.pose(
                    position=rep.distribution.uniform((-5,-10,0.8),(10,-8.5,1.0)),
                    scale=rep.distribution.uniform(15,20))
            with log3:
                rep.modify.pose(
                    position=rep.distribution.uniform((-10,-13,0.8),(2,10,1.0)),
                    scale=rep.distribution.uniform(15,20))
            with person1:
                rep.modify.pose(
                    position=rep.distribution.uniform((-3,-10,0.15),(1.2,5,0.4)),
                    scale=rep.distribution.uniform(1,1.2))
            with person2:
                rep.modify.pose(
                    position=rep.distribution.uniform((-2,-10,0.15),(10,-5,0.4)),
                    scale=rep.distribution.uniform(1,1.2))

            writer = rep.WriterRegistry.get("BasicWriter")
            writer.initialize(output_dir="_output",rgb=True,bounding_box_2d_loose=True)
            writer.attach([render_product])

            writer_top = rep.WriterRegistry.get("BasicWriter")
            writer_top.initialize(output_dir="_output_top", rgb=True)
            writer_top.attach([top_rp])

  # 매 프레임마다 그리드 맵을 생성하기 위해서 진행
    for i in range(frames):
        await rep.orchestrator.step_async()

        grid = generate_drone_grid_map(
            drone_prim_path=DRONE_PATH,
            map_center=(0.0, 0.0),
            map_size=(30.0, 30.0),
            resolution=0.05,
        )
        grid_person = generate_person_goal_map(
            person_root_path="/World/Scope/Person",
            map_center=(0.0, 0.0),
            map_size=(30.0, 30.0),
            resolution=2,
        )

        if grid_person is not None:
            p = person_dir / f"grid_map_person_{i:03d}.txt"
            print(f"[frame {i}] grid shape: {grid_person.shape}, obstacle cells: {int(grid_person.sum())}")
            np.savetxt(p, grid_person, fmt="%d")
            print(f"saved: grid_map_person_{i:03d}.txt")

        if grid is not None:
            p = obs_dir / f"grid_map_{i:03d}.txt"
            print(f"[frame {i}] grid shape:{grid.shape}, obstacle cells:{int(grid.sum())}")
            np.savetxt(p, grid, fmt="%d")
            print(f"saved: grid_map_{i:03d}.txt")

# 여기서 실행
asyncio.ensure_future(run(frames))
